<a href="https://colab.research.google.com/github/scalabrinig/cdProjetoAplicadoIV/blob/master/projeto/cd_projeto_aplicado_IV_entrega_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <img src="https://raw.githubusercontent.com/scalabrinig/cdProjetoAplicadoIV/d093146488f56dfcf0ef286bcee8efe0e71b9c76/figuras/mackenzie_logo.jpg" width="25%" align="right"/>

# **PROJETO APLICADO IV - Ciência de Dados EaD - 2026/01**


# **Entrega 1**

---

# **Titulo do Projeto**:

### *ChronoSec*: Detecção Comportamental e Séries Temporais para Segurança de Login (UEBA) em Eventos Google
---

In [1]:
#@title **Identificação do Grupo e Opção do Projeto**

#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe: \<nome\>, \<matrícula\>*)
Aluno1 = 'João Silveira Campos Netto, 10441670' #@param {type:"string"}
Aluno2 = 'Alex Luiz Rabelo, 10442968' #@param {type:"string"}

# **Introdução**

Processos de autenticação geram sinais ricos para segurança: falhas repetidas, desafios de MFA, mudanças de localização e padrões de horário. Em cenários de credenciais comprometidas, ataques automatizados e abuso de sessão, esses sinais costumam aparecer como anomalias temporais e comportamentais — muitas vezes antes de um incidente maior.

Este projeto propõe um produto analítico, implementado em Jupyter Notebooks, para explorar dados de login (anonimizados) provenientes da plataforma Google e aplicar técnicas de Séries Temporais e UEBA (User and Entity Behavior Analytics) para identificar desvios relevantes, como “Impossible Travel”, picos de frequência, sequências incomuns de eventos e mudanças no padrão de horário. A proposta combina regras explicáveis e modelos de aprendizado de máquina, com foco em reprodutibilidade e comunicação clara para um cliente (SOC/IAM).

UEBA é uma abordagem consolidada para detectar comportamentos anormais de usuários e entidades usando análises comportamentais e ML.



# **Motivações e justificativa**

### Motivação
1.	**Identidade virou o novo perímetro**

Grande parte dos incidentes modernos começa por abuso de credenciais (phishing, reutilização de senhas, token/session hijacking). O “momento do login” é um dos pontos mais ricos para detectar sinais iniciais de comprometimento, antes do dano maior.

2. **Logs de autenticação são séries temporais naturalmente**

Eventos de login têm ordem, frequência, dependência temporal e contexto (MFA, falha/sucesso, desafio), o que torna o problema ideal para técnicas de séries temporais e modelagem sequencial. Em vez de olhar eventos isolados, o projeto olha trajetórias de comportamento.

3. **Necessidade de reduzir ruído e acelerar triagem no SOC**

Times de segurança recebem muitos alertas, mas poucos são realmente relevantes. Um produto analítico que gere score + explicação ajuda a priorizar casos e reduzir tempo de investigação (MTTA/MTTR), com impacto direto na operação.

4. **“Impossible Travel” e padrões de sequência são sinais de alto valor**

Mudanças geográficas incompatíveis com o tempo, bursts de falhas e cadeias raras de autenticação são padrões amplamente utilizados na indústria, por serem interpretáveis e úteis na prática (mesmo antes do deep learning).

### Justificativa
1.	**Justificativa técnica: melhor modelar comportamento do que eventos pontuais**

Ataques de conta comprometida muitas vezes não são detectados por uma única característica. Eles aparecem como mudança de padrão: novo país/cidade, horário incomum, sequência anômala (falha→desafio→sucesso), ou velocidade impossível.

2. **Séries temporais (janelas, entropia, transições Markov, LSTM) são adequadas porque capturam:**
  - dependência temporal (antes/depois),
  - regularidade vs desvio (baseline do usuário),
  - anomalias contextuais (o que é normal para um usuário pode ser estranho para outro).

3. **Justificativa de negócio: prevenção e redução de impacto**

Detectar rapidamente abuso de credenciais reduz:
- fraudes e acessos indevidos,
- indisponibilidade por resposta a incidentes,
- custos de investigação e retrabalho,
- exposição reputacional e risco regulatório.

Além disso, gera um ativo “produto”: um motor de risco reaproveitável em outras aplicações (VPN, IAM, SSO, apps internos).

4. **Justificativa metodológica: abordagem em camadas (explicável + ML + deep learning)**

O projeto é justificável academicamente e operacionalmente porque entrega valor incremental:
- Regras explicáveis (rápidas de implementar e fáceis de defender em auditoria),
- Modelos clássicos (melhor generalização e redução de falsos positivos),
- Modelos sequenciais (Markov/LSTM) (capturam comportamento mais sutil).

Isso cria um caminho claro de maturidade do produto.

4. **Justificativa de dados: dataset apropriado e enriquecido**

Os dados anonimizados + geolocalização permitem medir:
- deslocamento (lat/lon + tempo),
- incerteza (accuracy_radius_km),
- contexto de autenticação (MFA/desafios/falhas),
- sinal auxiliar (É suspeito) para validação.

- Ou seja: há base suficiente para construir features robustas e validar hipóteses.

5. **Justificativa extensionista (ODS 9 e 12)**
- ODS 9: fortalecer resiliência e segurança de infraestrutura digital baseada em identidade.
- ODS 12: reduzir desperdício operacional (menos alertas inúteis, menos horas gastas, menos custos de resposta).

A entrega pública (código + dataset sintético + documentação) amplia o benefício para a comunidade acadêmica e profissionais de segurança.

# **Objetivo**

Este trabalho tem como objetivo desenvolver um produto analítico baseado em séries temporais para detectar e explicar comportamentos anômalos em processos de login (eventos de autenticação) a partir de dados anonimizados da plataforma Google enriquecidos com geolocalização. A pretensão central é transformar registros brutos de autenticação em informação acionável para Cyber Security, permitindo identificar rapidamente padrões compatíveis com abuso de credenciais, automação maliciosa e tentativas de acesso indevido, com foco em priorização operacional e redução de ruído na investigação.

Como metas, o projeto busca:
1. modelar o comportamento típico de usuários e domínios ao longo do tempo (baseline) e medir desvios relevantes;
2. implementar mecanismos de detecção para cinco dimensões complementares do problema: frequência/velocidade de tentativas (Velocity Checks), viagens impossíveis (Impossible Travel), sequências incomuns de eventos de autenticação via cadeias de Markov, mudanças no padrão de horário por entropia temporal, e aprendizado de comportamento por modelos LSTM;
3. consolidar essas evidências em um score de risco por evento acompanhado de justificativas interpretáveis, para apoiar triagem e tomada de decisão;
4. entregar um experimento executável e reproduzível em Jupyter Notebooks, incluindo pipeline de preparação de dados, extração de features, treinamento/avaliação de modelos e geração de relatórios com estudos de caso.

Ao final, espera-se disponibilizar uma solução que demonstre, de forma mensurável, a capacidade de identificar anomalias comportamentais e explicar por que um evento foi considerado suspeito, além de apresentar uma proposta extensionista alinhada às ODS 9 e 12 por meio da publicação do método e de materiais reprodutíveis (incluindo um dataset compatível com o esquema de dados), contribuindo para a comunidade interessada em segurança de identidade e análise temporal.



# **Descrição da base de dados**

Apresentar a descrição do conjunto de dados que será usado no trabalho e apontando quais informações estão disponíveis, sua estrutura e organização, sua forma e seu período de coleta, além da indicação da fonte de dados.

# **Referências**

Expor a relação dos recursos utilizados na elaboração do documento, desde que tenham sido devidamente citados. Para isso, devem ser utilizadas as normas da ABNT.


In [2]:
#@title **Avaliação**
Introducao = 10 #@param {type:"slider", min:0, max:10, step:1}

Fonte_dos_dados = 10 #@param {type:"slider", min:0, max:10, step:1}

Solucao_proposta = 10 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 10 #@param {type:"slider", min:0, max:10, step:1}

In [5]:
#@title **Nota Final**
nota = 0.30*Introducao + 0.30*Fonte_dos_dados + 0.30*Solucao_proposta + 0.10*Referencias

print(f'Nota da entrega {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_nome = []

for i in range(1,3):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_nome.append(lista[0]);")

alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota da entrega 10.0



,nome,nota
0,João Silveira Campos Netto,10.0
1,Alex Luiz Rabelo,10.0
